## Caudales extremos en Chile

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#Cargamos el dataset
dir = "desafio_spike_cuencas-master/caudal_extra.csv"
df = pd.read_csv(dir)

In [ ]:
#Cantidad de nulls
df.isnull().sum()

### Análisis dataset

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.describe()

Vemos que las columnas correspondientes a precip_promedio y temp_max_promedio tienen menor valor en conteo que el n° de filas del data set, ahí tenemos missing values. Esto probablemente ocurre porque hay cuencas sin estaciones de temperatura o precipitación y por lo tanto nunca se adquirieron esos datos.

In [ ]:
df.columns.values.tolist()

In [ ]:
#Eliminaremos algunas columnas del dataset que puedan representar información redundante o que no añadan nada 
#relevante
drop = list(["Unnamed: 0","institucion","fuente","codigo_cuenca","gauge_name",])

In [ ]:
df = df.drop(drop,axis=1)

In [ ]:
df.shape

In [ ]:
df.columns.values

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#Gráficos de dispersión para observar correlación entre variables

#Relación caudal con precipitación promedio
df.plot(kind="scatter",x="precip_promedio",y="caudal")

Aquí pareciera haber una tendencia a que a medida que la precipitación promedio aumenta, el caudal disminuye,
o equivalentemente, a medida que la precipitación promedio disminuye el caudal aumenta. Esto podría ser porque 
al cambiar de estación invernal a primavera las lluvias disminuyen pero comienza a derretirse la nieve, con 
la consecuencia de que aumentan los caudales.

In [ ]:
#Relación caudal con temperatura máxima promedio
df.plot(kind="scatter",x="temp_max_promedio",y="caudal")

In [ ]:
df.plot(kind="scatter",x="temp_max_promedio",y="precip_promedio")

In [ ]:
#Histogramas
bins = int(np.ceil(1+np.log2(1411180))) #divisiones histograma
#Caudal
plt.hist(df["caudal"],bins=bins)
#data = df[df["year"]==2000]
#plt.hist(data["caudal"])
plt.xlabel("Caudal")
plt.ylabel("Frecuencia")
plt.title("Histograma de caudal")



In [ ]:
#Precipitación promedio
plt.hist(df["precip_promedio"].dropna(),bins=bins)
plt.xlabel("Precipitación promedio")
plt.ylabel("Frecuencia")
plt.title("Histograma de precipitación promedio")

In [ ]:
#Temperatura máxima promedio
plt.hist(df["temp_max_promedio"].dropna(),bins=bins)
plt.xlabel("Temperatura máxima promedio")
plt.ylabel("Frecuencia")
plt.title("Histograma de temperatura máxima promedio")

El caudal y la precipitacion promedio se concentran en ciertos valores, en el caso de caudal promedio
los valores se concentran entre 0 y 500 aprox, mientras que en el caso de la precipitacion promedio esta 
se concentra entrte 0 y 2 mm, lo cual tiene sentido porque hay más días con poca lluvia o sin lluvia.
Otro dato a considerar es el máximo de precipitacíón en un día, el cual corrsponde a 258 mm.

Las temperaturas máximas promedio se distribuyen de una forma que tiene a la distribución normal
aunque de manera aproximada, el promedio está en torno a los 15°.



In [ ]:
#Reemplazaremos los missing values con el valor más cercano que encuentre
#al missing value, ya que las mediciones son en tiempos cercanos y se 
#espera que por la temporada sean valores cercanos
#Precipitación promedio 
df["precip_promedio"] = df["precip_promedio"].fillna(method="ffill")
#Temperatura max 
df["temp_max_promedio"] = df["temp_max_promedio"].fillna(method="ffill")

In [ ]:
#Quedan algunos nan en temp_max_promedio, los reemplazaremos por la media.
df["temp_max_promedio"] = df["temp_max_promedio"].fillna(df["temp_max_promedio"].mean())

In [ ]:
#Lugares donde están las cuencas .... que se están estudiando
cuencas_dist = df.nombre.unique()
cuencas_dist

### Plots precipitación, temperatura, caudal

In [ ]:
#Antes de crear la función veremos qué grupos hay según cada código de
#estación
num = df.codigo_estacion.unique()
print("Hay "+str(len(num))+" estaciones distintas")
group_codest = df.groupby("codigo_estacion")

In [ ]:
for estacion,grupos in group_codest:
    print(estacion)
    print(grupos)
    

In [ ]:
#Convertiremos la columna fecha de str a objeto datetime y crearemos 
#una columna donde solo se indique el año.


In [ ]:
df["fecha"] = pd.to_datetime(df["fecha"])

In [ ]:
#Nueva columna
df["year"] = df["fecha"].dt.year

In [ ]:
#Función para hacer plot de una columna

def time_plot_una_estacion(codigo_estacion, columna, fecha_min, fecha_max):
    #Ingresar fechas como string en formato 'yyyy-mm-dd'
    #codigo_estacion es un numero, columna ingresarlo como string
    import matplotlib.pyplot as plt
    import matplotlib.ticker as tkr
    import matplotlib.dates as mdates
    import datetime

    #Defino fechas para graficar
    start_date = pd.to_datetime(fecha_min)
    end_date = pd.to_datetime(fecha_max)
    
    #Filtro segun codigo_estacion y tomo valores entre las fechas 
    #indicadas
    data = df[(df["codigo_estacion"]==codigo_estacion)&
        (df["fecha"]>=start_date) & (df["fecha"]<=end_date)]
    #Ordenamos filas según fecha
    data2 = data.sort_values(by=["fecha"])

    plt.figure(figsize=(20,4))
    plt.plot(data2["fecha"],data2[columna])
    plt.xlabel('Fecha')
    plt.ylabel(str(columna))

In [ ]:
#Probando función
#Códigos para probar: 4540001 (1960-1984),10414001 11040001
time_plot_una_estacion(4540001,"precip_promedio",'1960-01-01','1960-12-31')

In [ ]:
#Función para hacer plot de varias columnas

def time_plot_estaciones_varias_columnas(codigo_estacion, columnas, fecha_min, fecha_max):
    #Ingresar fechas como string en formato 'yyyy-mm-dd'
    #codigo_estacion es un numero
    #columnas ingresarlas en el formato: ["columna1","columna2",etc]
    import matplotlib.pyplot as plt
    import matplotlib.ticker as tkr
    import matplotlib.dates as mdates
    import datetime

    #Defino fechas para graficar
    start_date = pd.to_datetime(fecha_min)
    end_date = pd.to_datetime(fecha_max)
    
    #Filtro segun codigo_estacion y tomo valores entre las fechas 
    #indicadas
    data = df[(df["codigo_estacion"]==codigo_estacion)&
        (df["fecha"]>=start_date) & (df["fecha"]<=end_date)]
    #Ordenamos filas según fecha
    data2 = data.sort_values(by=["fecha"])

    #Normalizo las columnas antes de graficar
    col1 = data2[columnas[0]]/(max(data2[columnas[0]])-min(data2[columnas[0]]))
    col2 = data2[columnas[1]]/(max(data2[columnas[1]])-min(data2[columnas[1]]))
    col3 = data2[columnas[2]]/(max(data2[columnas[2]])-min(data2[columnas[2]]))
    
 
    plt.figure(figsize=(20,4))
    plt.plot(data2["fecha"],col1)
    plt.plot(data2["fecha"],col2)
    plt.plot(data2["fecha"],col3)
    plt.xlabel('Fecha')
    #plt.ylabel(str(columnas))
    plt.legend()

In [ ]:
#Probando función
#Códigos para probar: 4540001 (1960-1984),11335002 (2017)
time_plot_estaciones_varias_columnas(11335002,["caudal","precip_promedio","temp_max_promedio"],'2017-01-01','2017-12-31')

### Variables nuevas

Crearemos una nueva columna para indicar la estación, considerando:
* Otoño: 21/marzo - 20/junio
* Invierno: 21/junio - 20/septiembre
* Primavera: 21/septiembre - 20/Diciembre
* Verano: 21/Diciembre - 20/marzo

In [ ]:
#Máscaras para cada estación del año
mask_otoño = (df.fecha.dt.month==4) | (df.fecha.dt.month==5) | ((df.fecha.dt.month==3) & 
            (df.fecha.dt.day>=21)) | ((df.fecha.dt.month==6) & (df.fecha.dt.day<=21))
mask_invierno = (df.fecha.dt.month==7) | (df.fecha.dt.month==8) | ((df.fecha.dt.month==6)&
            (df.fecha.dt.day>=21)) | ((df.fecha.dt.month==9)&(df.fecha.dt.day<=20))
mask_primavera = (df.fecha.dt.month==10) | (df.fecha.dt.month==11) | ((df.fecha.dt.month==9)
    &(df.fecha.dt.day>=21)) | ((df.fecha.dt.month==12)&(df.fecha.dt.day<=20))
mask_verano = (df.fecha.dt.month==1) | (df.fecha.dt.month==2) | ((df.fecha.dt.month==12)
    &(df.fecha.dt.day>=21)) | ((df.fecha.dt.month==3)&(df.fecha.dt.day<=20))

In [ ]:
#Creo la columna con las estaciones
df["estaciones"] = 0
df["estaciones"].loc[mask_otoño] = "otoño"
df["estaciones"].loc[mask_invierno] = "invierno"
df["estaciones"].loc[mask_primavera] = "primavera"
df["estaciones"].loc[mask_verano] = "verano"

In [ ]:
df.head(3)

In [ ]:
cuencas = df.groupby("codigo_estacion")
cuencas_groups = cuencas.groups
keys = cuencas_groups.keys()   #con esto tengo los códigos de todas las estaciones de medición de caudal
estaciones_key = ["otoño","invierno","primavera","verano"]
df["caudal_extremo"] = 0
for cuencas in keys:
    for est in estaciones_key:
        caudales = df.caudal.loc[(df.codigo_estacion==cuencas)&(df.estaciones==est)]
        p95caudal = np.percentile(caudales,95)
        #df["caudal_extremo"] = 0
        df["caudal_extremo"].loc[(df["codigo_estacion"]==cuencas)&(df["estaciones"]==est)&(df["caudal"]>p95caudal)] = 1
  

In [ ]:
num_cuencas = df.gauge_id.unique()

In [ ]:
len(num_cuencas)

In [ ]:
#cuencas = df.groupby("codigo_estacion")
#cuencas_groups = cuencas.groups
#cuencas_groups.keys()  

In [ ]:
#temp_extremo
df["temp_extremo"] = 0
for cuencas in keys:
    for est in estaciones_key:
        temps = df.temp_max_promedio.loc[(df.codigo_estacion==cuencas)&(df.estaciones==est)]
        p95temp = np.percentile(temps,95)
        df["temp_extremo"].loc[(df["codigo_estacion"]==cuencas)&(df["estaciones"]==est)&(df["temp_max_promedio"]>p95temp)] = 1
  

In [ ]:
#precip_extremo
df["precip_extremo"] = 0
for cuencas in keys:
    for est in estaciones_key:
        precip = df.precip_promedio.loc[(df.codigo_estacion==cuencas)&(df.estaciones==est)]
        p95precip = np.percentile(precip,95)
        df["precip_extremo"].loc[(df["codigo_estacion"]==cuencas)&(df["estaciones"]==est)&(df["precip_promedio"]>p95precip)] = 1

In [ ]:
df.head(100)

### Variabel caudal_extremo

In [ ]:
keys

In [ ]:
time_plot_una_estacion(1020003, "caudal_extremo", '1960-01-01', '2018-12-12')
time_plot_una_estacion(1021001, "caudal_extremo", '1960-01-01', '2018-12-12')
time_plot_una_estacion(2110004, "caudal_extremo", '1960-01-01', '2018-12-12')
time_plot_una_estacion(5423003, "caudal_extremo", '1960-01-01', '2018-12-12')
time_plot_una_estacion(8132001, "caudal_extremo", '1960-01-01', '2018-12-12')

Graficando en algunas cuencas lo que se observa es que los caudales extremos entre una y otra cuenca están poco
relacionados, al menos observando a lo largo de los años. Si se observara para un año en particular:

In [ ]:
time_plot_una_estacion(1020003, "caudal_extremo", '2000-01-01', '2000-12-12')
time_plot_una_estacion(1021001, "caudal_extremo", '2000-01-01', '2000-12-12')
time_plot_una_estacion(2110004, "caudal_extremo", '2000-01-01', '2000-12-12')
time_plot_una_estacion(5423003, "caudal_extremo", '2000-01-01', '2000-12-12')
time_plot_una_estacion(8132001, "caudal_extremo", '2000-01-01', '2000-12-12')

A lo largo de un año también son muy variables los caudales extremos comparando entre cuencas.

In [ ]:
## Agrupamos el dataset de acuerdo a los canales registrados en la variable gauge_id y calculamos el porcentaje 
#de caudales extremos de cada cuenca 

In [ ]:
#Primero haremos una prueba manual calculando el porcentaje para une cuenca
df[df['gauge_id']==1020003][['gauge_id','caudal_extremo']]

In [ ]:
porcentaje_test = sum(df[df['gauge_id']==1020003]['caudal_extremo'])/len(df[df['gauge_id']==1020003]['caudal_extremo'])
porcentaje_test

In [ ]:
#Realizaremos operaciones sobre cada grupo (cuenca) por separado, en vez de usar una operación sobre todas las filas
df.groupby('gauge_id')["caudal_extremo"].aggregate(
    {
    "Registros Caudales Extremos": np.sum,
    "Total Registros": lambda h: len(h),
    "Porcentaje": lambda h:np.sum(h)/len(h)*100
    }
)

In [ ]:
#La cantidad de eventos de caudal extremo ronda el 5% en la mayoría de las cuencas, los comportamientos en las 
#diferentes cuencas son similares.

### Porcentaje de eventos extremos a través del tiempo

In [ ]:
df.head()

In [ ]:
#Creamos la columna year
df["year"] = df["fecha"].dt.year


In [ ]:
porcentaje_temp_extrema_year = df.groupby("year")['temp_extremo'].aggregate(
{
    "Porcentaje_temperatura_extrema": lambda h:np.sum(h)/len(h)*100
}
)

In [ ]:
porcentaje_temp_extrema_year.head(3)

In [ ]:
porcentaje_temp_extrema_year.size

In [ ]:
#Gráfico de barras
plt.bar(porcentaje_temp_extrema_year.index,porcentaje_temp_extrema_year.Porcentaje)

In [ ]:
porcentaje_caudal_extremo_year = df.groupby("year")['caudal_extremo'].aggregate(
{
    "Porcentaje_caudal_extremo": lambda h:np.sum(h)/len(h)*100
}
)
porcentaje_precip_extrema_year = df.groupby("year")['precip_extremo'].aggregate(
{
    "Porcentaje_precipitacion_extrema": lambda h:np.sum(h)/len(h)*100
}
)

In [ ]:
barWidth = 0.4
r1 = np.arange(len(porcentaje_temp_extrema_year.index))
r2 = [x + barWidth for x in r1]
index_plot = [r +0.2  for r in range(len(porcentaje_temp_extrema_year.index))]

In [ ]:
plt.figure(figsize=(18,8))
plt.plot(index_plot,porcentaje_caudal_extremo_year.Porcentaje_caudal_extremo,color="blue")
plt.bar(r1,porcentaje_temp_extrema_year.Porcentaje_temperatura_extrema,color="magenta",label='Temperatura',align='center',width = 0.4)
plt.bar(r2,porcentaje_precip_extrema_year.Porcentaje_precipitacion_extrema,color="gray",label='Precipitaciones',align='center',width = 0.4)

plt.xticks(index_plot,porcentaje_temp_extrema_year.index,rotation=90)
plt.legend()


In [ ]:
plt.plot(porcentaje_temp_extrema_year.index,porcentaje_caudal_extremo_year.Porcentaje_caudal_extremo)
plt.plot(porcentaje_temp_extrema_year.index,porcentaje_temp_extrema_year.Porcentaje_temperatura_extrema)
plt.plot(porcentaje_temp_extrema_year.index,porcentaje_precip_extrema_year.Porcentaje_precipitacion_extrema)
plt.legend()